In [11]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import base64

print("TF Version: {}".format(tf.__version__))
print("Numpy Version: {}".format(np.__version__))

TF Version: 1.14.0
Numpy Version: 1.17.3


# Model

## Numeric Example

In [12]:
def model_fn(features, labels, mode, params):
    
    _x = features["x"]
    
    if mode in [tf.estimator.ModeKeys.TRAIN, tf.estimator.ModeKeys.EVAL]:
        pass
    
    # build a non-linear model
    with tf.variable_scope("n1"):
        a = tf.get_variable("a", [1,1], initializer=tf.random_normal_initializer(mean=1.0, stddev=1.0))
    with tf.variable_scope("n2"):
        b = tf.get_variable("b", [1,1], initializer=tf.random_normal_initializer(mean=1.0, stddev=1.0))
    with tf.variable_scope("n3"):
        c = tf.get_variable("c", [1,1], initializer=tf.random_normal_initializer(mean=1.0, stddev=1.0))
    with tf.variable_scope("output"):
        logit = tf.matmul(tf.square(_x), a) + tf.matmul(_x, b) + c
    
    # define the result
    result = tf.identity(logit, name="result")
    predict_dict = {"result": result}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predict_dict)
    
    # define loss function
    loss = tf.reduce_sum(tf.square(tf.subtract(labels, logit)))
    
    # define optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])
    train_opt = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    # metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.mean_squared_error(labels, logit)
    }
    
    # return an estimator spec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_opt,
        eval_metric_ops=eval_metric_ops)

In [13]:
def func(x):
  return 3*x*x + 2*x - 1 

def train_generator(start=-1, end=1, batch_size=200):
  scaled = batch_size*10
  _x = np.random.randint(start*scaled, end*scaled, batch_size) / scaled
  _y = np.array(list(map(func, _x)))
  x = _x.reshape(-1, 1).astype('float32')
  y = _y.reshape(-1, 1).astype('float32')
  return x, y

In [14]:
total_size=5000
x_train, y_train = train_generator(start=-5, end=5, batch_size=total_size)
x_train[0], y_train[0], x_train.shape, y_train.shape

(array([3.11108], dtype=float32),
 array([34.258617], dtype=float32),
 (5000, 1),
 (5000, 1))

In [15]:
batch_size = 64
epochs = total_size // batch_size

model_params = {"learning_rate": 1e-4, "batch_size": batch_size}

In [16]:
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x139ea8c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
for _ in range(5):
    x_train, y_train = train_generator(start=-5, end=5, batch_size=total_size)

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x_train},
        y=y_train,
        num_epochs=epochs,
        shuffle=True, 
        batch_size=batch_size)

    # Train
    nn.train(input_fn=train_input_fn, steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v/model.ckpt.
INFO:tensorflow:loss = 82300.17, step = 1
INFO:tensorflow:global_step/sec: 885.472
INFO:tensorflow:loss = 116.04344, step = 101 (0.113 sec)
INFO:tenso

INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v/model.ckpt.
INFO:tensorflow:loss = 5.273961e-10, step = 5001
INFO:tensorflow:global_step/sec: 940.035
INFO:tensorflow:loss = 4.3176573e-10, step = 5101 (0.107 sec)
INFO:tensorflow:global_step/sec: 1024.99
INFO:tensorflow:loss = 9.571921e-10, step = 5201 (0.098 sec)
INFO:tensorflow:global_step/sec: 1392.48
INFO:tensorflow:loss = 7.82983e-10, step = 5301 (0.072 sec)
INFO:tensorflow:global_step/sec: 1273.51
INFO:tensorflow:loss = 1.2580108e-09, step = 5401 (0.078 sec)
INFO:tensorflow:global_step/sec: 1164.95
INFO:tensorflow:loss = 7.362729e-10, step = 5501 (0.086 sec)
INFO:tensorflow:global_step/sec: 997.722
INFO:

INFO:tensorflow:loss = 5.1197024e-10, step = 11801 (0.087 sec)
INFO:tensorflow:global_step/sec: 962.01
INFO:tensorflow:loss = 4.960406e-10, step = 11901 (0.104 sec)
INFO:tensorflow:global_step/sec: 1002.27
INFO:tensorflow:loss = 8.6274454e-10, step = 12001 (0.100 sec)
INFO:tensorflow:global_step/sec: 963.818
INFO:tensorflow:loss = 3.4877318e-10, step = 12101 (0.104 sec)
INFO:tensorflow:global_step/sec: 1172.13
INFO:tensorflow:loss = 7.977832e-10, step = 12201 (0.086 sec)
INFO:tensorflow:global_step/sec: 1204.98
INFO:tensorflow:loss = 3.1760372e-10, step = 12301 (0.082 sec)
INFO:tensorflow:global_step/sec: 1234.09
INFO:tensorflow:loss = 3.9282652e-10, step = 12401 (0.081 sec)
INFO:tensorflow:global_step/sec: 1310.27
INFO:tensorflow:loss = 4.877627e-10, step = 12501 (0.078 sec)
INFO:tensorflow:global_step/sec: 1175.5
INFO:tensorflow:loss = 3.0328742e-10, step = 12601 (0.084 sec)
INFO:tensorflow:global_step/sec: 950.344
INFO:tensorflow:loss = 7.0338585e-10, step = 12701 (0.106 sec)
INFO:t

INFO:tensorflow:loss = 6.96217e-10, step = 18901 (0.091 sec)
INFO:tensorflow:global_step/sec: 1007.3
INFO:tensorflow:loss = 4.6119752e-10, step = 19001 (0.099 sec)
INFO:tensorflow:global_step/sec: 981.48
INFO:tensorflow:loss = 5.662591e-10, step = 19101 (0.102 sec)
INFO:tensorflow:global_step/sec: 965.662
INFO:tensorflow:loss = 5.631142e-10, step = 19201 (0.105 sec)
INFO:tensorflow:global_step/sec: 953.815
INFO:tensorflow:loss = 1.3266757e-09, step = 19301 (0.103 sec)
INFO:tensorflow:global_step/sec: 1252.52
INFO:tensorflow:loss = 3.6791392e-10, step = 19401 (0.080 sec)
INFO:tensorflow:global_step/sec: 1145.52
INFO:tensorflow:loss = 5.315552e-10, step = 19501 (0.087 sec)
INFO:tensorflow:global_step/sec: 1107.61
INFO:tensorflow:loss = 4.514753e-10, step = 19601 (0.090 sec)
INFO:tensorflow:global_step/sec: 922.691
INFO:tensorflow:loss = 5.1419136e-10, step = 19701 (0.108 sec)
INFO:tensorflow:global_step/sec: 1238.25
INFO:tensorflow:loss = 3.2060263e-10, step = 19801 (0.081 sec)
INFO:tens

In [18]:
for n in nn.get_variable_names():
    print(n, nn.get_variable_value(n))

global_step 25000
n1/a [[3.0000002]]
n2/b [[2.]]
n3/c [[-0.99999994]]


In [19]:
pred_data = np.array([1,2]).astype('float32').reshape([-1, 1])
pred_data.shape

(2, 1)

In [20]:
# Print out predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": pred_data},
    num_epochs=1,
    shuffle=False)

import struct
predictions = nn.predict(input_fn=predict_input_fn)
for _, p in enumerate(predictions):
    print(p["result"])

print(list(map(func, pred_data)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[4.]
[15.]
[array([4.], dtype=float32), array([15.], dtype=float32)]


In [21]:
def serving_input_receiver_fn_2():
    """Serving input_fn that builds features from placeholders"""
    
    # if you use tf.placeholder() as the serialized examples with a parse_example() processing function
    # the data type in placeholder() must be string due to receiving tf.train.Example()
    serialized_tf_example = tf.placeholder(tf.string, shape=[None])
    receiver_tensors = {'examples': serialized_tf_example}
    
    # definition to features must be the same with data input in trainer (tf.estimator.inputs.numpy_input_fn)
    features = tf.io.parse_example(serialized_tf_example, 
                                  {'x': tf.FixedLenFeature(shape=[1], dtype=tf.float32)})
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

def serving_input_receiver_fn():
    """A simple way to build a receiver function for serving a model."""
    inputs = tf.placeholder(tf.float32, shape=[None, 1])
    return tf.estimator.export.ServingInputReceiver({"x": inputs}, inputs)

print(serving_input_receiver_fn_2())
print(serving_input_receiver_fn())

ServingInputReceiver(features={'x': <tf.Tensor 'ParseExample/ParseExample:0' shape=(?, 1) dtype=float32>}, receiver_tensors={'examples': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>}, receiver_tensors_alternatives=None)
ServingInputReceiver(features={'x': <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>}, receiver_tensors={'input': <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>}, receiver_tensors_alternatives=None)


In [22]:
if os.path.exists("/Users/jiankaiwang/Desktop/TFE_SavedModel/"):
    exported_name = nn.export_saved_model("/Users/jiankaiwang/Desktop/TFE_SavedModel/", serving_input_receiver_fn)
else:
    exported_name = nn.export_saved_model("/tmp/TFE_SavedModel/", serving_input_receiver_fn)
    
outmodel = exported_name.decode("UTF-8")
print(outmodel)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /var/folders/4b/wh1nthj1563b1gl58ydgwnfw0000gp/T/tmpba7pke4v/model.ckpt-25000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /Users/jiankaiwang/Desktop/TFE_SavedModel/temp-b'1575452753'/saved_model.pb
/Users/jiankaiwang/Desktop/TFE_SavedModel/1575452753


In [23]:
!saved_model_cli show --dir /Users/jiankaiwang/Desktop/TFE_SavedModel/1575452753 --tag_set serve --signature_def serving_default

/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jiankaiwang/devops/pyenv/tfx/lib/pyt

# Example

Instantiate a container first before calling it via a restful API.

```sh
# instantiate a new container.
docker run --name serving -p 8501:8501 -e MODEL_NAME=TFE_SavedModel tensorflow/serving &

# copy the folder containing a new model into the container
docker cp TFE_SavedModel/1575452753 serving:/models/TFE_SavedModel
```

## Requests via Restful APIs

In [9]:
import json
import requests

In [26]:
# prepare a simple example
my_data = {"instances": [[1.1], [2.0], [1.0]]}

# the serving url
serving_url = "http://localhost:8501/v1/models/TFE_SavedModel:predict"

# a post request
r = requests.post(serving_url,
                  data=json.dumps(my_data),
                  headers={'Content-Type': 'application/octet-stream'})
print(r.status_code, r.content)

200 b'{\n    "predictions": [[4.8300004], [15.0], [4.0]\n    ]\n}'
